# Channel Turbulent with 2 groupes IATE - Validation

# Description

This document aims to validate the source terms of the two-group IATE by reproducing the experiment described in "Interfacial Structures in Confined Cap-Turbulent and Churn-Turbulent Flows" by X. Sun et al. (2004, https://doi.org/10.1016/j.ijheatfluidflow.2003.08.001). The experiment covers 13 flow conditions within cap-turbulent and churn-turbulent regimes, conducted in a vertical air-water two-phase flow loop with upward flow however only Run 13 is computed here. The test section measures 200 mm in width and has a 10 mm gap. Bubble characteristics, captured by probes, are classified into two groups as for the two-group IATE: spherical or distorted bubbles (Group 1) and cap or churn-turbulent bubbles (Group 2).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
matplotlib.rcParams['text.usetex'] = True
import copy
import os, sys
from string import Template
import time
from IPython.display import display
import math

from trustutils import run 
from trustutils import visit
from trustutils.jupyter import plot

plt.rcParams["font.weight"] = "bold"

run.TRUST_parameters()



# Definition of probes

In [ ]:
sondes = { }

n_seg_sonde = 50
r_tube = 0.100

tab_name_sondes = ["k",
                   "omega",
                   "vitesse_liquide_eau",
                   "vitesse_gaz_air_group1",
                   "vitesse_gaz_air_group2",
                   "alpha_gaz_air_group1",
                   "alpha_gaz_air_group2",
                   "pression",
                   "dp",
                   "lift",
                   "disp",
                   "vorticite",
                   "d_bulles",
                   "interfacial_area"
                  ]

x_0 = 0.00125
y_0 = 0.
x_1 = 0.0025
y_1 = 0.
x_2 = 0.00375
y_2 = 0.

x_w0 = 0.00125
y_w0 = 0.100
x_w1 = 0.0025
y_w1 = 0.100
x_w2 = 0.00375
y_w2 = 0.100

z88 = 0.01905*(88-35)
z142 = 0.01905*(142-35)

str_sonde = " "
for name_sonde in tab_name_sondes:
    str_sonde += f" {name_sonde}_881 {name_sonde} periode 1.e-2 segment {n_seg_sonde} {x_0} {y_0} {z88} {x_w0} {y_w0} {z88}  \n"
    str_sonde += f" {name_sonde}_882 {name_sonde} periode 1.e-2 segment {n_seg_sonde} {x_1} {y_1} {z88} {x_w1} {y_w1} {z88}  \n"
    str_sonde += f" {name_sonde}_883 {name_sonde} periode 1.e-2 segment {n_seg_sonde} {x_2} {y_2} {z88} {x_w2} {y_w2} {z88}  \n"
    str_sonde += f" {name_sonde}_1421 {name_sonde} periode 1.e-2 segment {n_seg_sonde} {x_0} {y_0} {z142} {x_w0} {x_w0} {z142}  \n"
    str_sonde += f" {name_sonde}_1242 {name_sonde} periode 1.e-2 segment {n_seg_sonde} {x_1} {y_1} {z142} {x_w1} {y_w1} {z142}  \n"
    str_sonde += f" {name_sonde}_1423 {name_sonde} periode 1.e-2 segment {n_seg_sonde} {x_2} {y_2} {z142} {x_w2} {y_w2} {z142}  \n"

sondes["PolyMAC"] = str_sonde

# Preparation and launching of the computation

In [ ]:
force_recalculation = True 
number_of_partitions = 4

# Mesh
schema = { "PolyMAC" : " PolyMAC_P0  dis option_PolyMAC { } " }
dimension = {  "PolyMAC" : 3}
solveur = { "ICE" : ""}
paroi = {   "PolyMAC" : " paroi "}
paroi_adiabatique = { "PolyMAC" : " paroi_adiabatique "}

# Numerical parameters
facsec  = 0.1
tmax    = 100
nb_pas_dt_max = '10000'
seuil_statio  = '1.e-3'

#Physical quantities
air_properties_1 =   "     gaz_air_group1 Fluide_Incompressible { mu champ_uniforme 1 1.e-5 rho champ_uniforme 1  1.   lambda Champ_Uniforme  1 0.023 Cp Champ_Uniforme 1 1006   beta_th Champ_Uniforme 1 0 } "
air_properties_2 =   "     gaz_air_group2 Fluide_Incompressible { mu champ_uniforme 1 1.e-5 rho champ_uniforme 1  1.   lambda Champ_Uniforme  1 0.023 Cp Champ_Uniforme 1 1006   beta_th Champ_Uniforme 1 0 } "
water_properties = " liquide_eau Fluide_Incompressible { mu champ_uniforme 1 1.e-3 rho champ_uniforme 1  1.e3 lambda Champ_Uniforme  1 0.604 Cp Champ_Uniforme 1 75.366 beta_th Champ_Uniforme 1 0 } "
interface =        "interface_eau_air interface_sigma_constant  { tension_superficielle 0.0728 }"

# Steady state target
u_0 = 4.853 
alpha_cible = 0.415
alpha_cible1 = 0.075
alpha_cible2 = 0.191


#Turbulent quantities
D_h = 0.01905
CI_k  = 0.01*u_0**2
CI_om = CI_k**0.5*0.09**0.5/0.42/D_h/0.1
        
#Two-phase quantities
frottement_interfacial = " Tomiyama_complet { contamination 2 } " 
portance_interfaciale = " Tomiyama_complet { }" 
dispersion_bulles = " turbulente_Bazin { } " 
db1 = 0.00316
db2 = 0.0174
beta_portance = "1"
beta_disp     = "1"
Cw1 = -.1
Cw2 = .147

# Guess of inlet conditions for target
Cd1 = 2./3.* db1 /0.00275 
Cd2 = 8./3.
u_r1 = np.sqrt(4.*db1/(3*Cd1)*9.81*(1000.-1.)/1000.*(1.-alpha_cible))
u_r2 = np.sqrt(4.*db2/(3*Cd2)*9.81*(1000.-1.)/1000.*(1.-alpha_cible))
alpha_entree1 = alpha_cible1 *(u_0+u_r1)/u_0
alpha_entree2 = alpha_cible2 *(u_0+u_r2)/u_0
ai1 = 6*alpha_entree1/db1
ai2 = 6*alpha_entree2/db2

# Building computation
build_dir = run.BUILD_DIRECTORY

if force_recalculation or not os.path.exists(f'{build_dir}'):
    print('Recalculation running..')
    run.reset()

    for s in schema.keys() :
        for solv in solveur.keys() :
            name = f"{s}_{solv}"

            substitutions_dict = {"dimension" : dimension[s],
                                    "schema" : schema[s],

                                    "tmax": str(tmax),                                  
                                    "facsec" : str(facsec) ,
                                    "nb_pas_dt_max" : nb_pas_dt_max ,
                                    "seuil_statio" : seuil_statio ,
                                    "solveur" : solv ,
                                    "facsec_diffusion_for_sets" : solveur[solv] ,
                                                
                                    "carrying_phase" : water_properties ,
                                    "dispersed_phase_1" : air_properties_1 ,
                                     "dispersed_phase_2" : air_properties_2 ,
                                    "interface" : interface,
                                                
                                    "diametre_bulles" : str(db1),
                                    "u_0"  : f"{3*dimension[s]} {''.join([ ' 0 ' for i in range(3*(dimension[s]-1))])} {u_0} {u_0}*1.53 {u_0}*1.53 " ,
                                    "grav"   : f"{3*dimension[s]} {''.join([ ' 0 ' for i in range(3*(dimension[s]-1))])} -9.81 -9.81 -9.81 " ,
                                    "alpha_l0" : str(1-alpha_entree1-alpha_entree2) ,
                                    "alpha_v01" : str(alpha_entree1) ,
                                    "alpha_v02" : str(alpha_entree2) ,
                                    "frottement_interfacial" : frottement_interfacial,
                                    "portance_interfaciale" : portance_interfaciale,
                                    "dispersion_bulles" : dispersion_bulles,
                                    "beta_portance" : beta_portance,
                                    "beta_disp" : beta_disp,
                                                
                                    "CI_diss": f"{CI_om}" ,
                                    "CI_k": f"{CI_k}" ,
                                    "paroi_adiabatique" : paroi_adiabatique[s],
                                    "paroi" : paroi[s],

                                    "ci_ai1" : str(ai1),
                                    "ci_ai2" : str(ai2),

                                    "sondes" : sondes[s],
                                                
                                }

            tc = run.addCaseFromTemplate("jdd.data",
                                         targetDirectory=f"{name}",
                                         dic=substitutions_dict,
                                         nbProcs=number_of_partitions)
            if run.isExtractingNR():
                tc.substitute("Nombre_de_Noeuds 4 51 351","Nombre_de_Noeuds 4 20 140")

            if number_of_partitions > 1:
                tc.partition()

    run.printCases()
    run.runCases()
    display(run.tablePerf())

else:
    print('No recalculation, reusing old results!')

# Loading results

In [ ]:
############# results : sim[config][debit air][raf][position]
#############         columns : ['y/h'] ['vxl'] ['vyl'] ['vxg'] ['vyg'] ['k'] ['alp'] ['DB']
#############                   ["diff"] ["conv"] ["press"] ["grav"] ["drag"] ["lift"] ["disp"] ["lub"]
#############               nb : end column name with lx for forces
############# sim name : tab_name [config][debit air][raf]
############# sim label: tab_label[config][debit air][raf]
############# convergence :  dt_ev[config][debit air][raf]
#############         columns : ['time']['dt']['facsec']['residu']['dt_stab']['vitl']['vitg']['alphal']['alphag']['Tl']['Tg']['diss']['k'] 

sim      = {  }
dt_ev    = {  }
par = ""
if number_of_partitions > 1 : par = "PAR_"

for s in schema.keys() :
    for solv in solveur.keys() :
        name = f"{s}_{solv}"

        dt_ev[name] = pd.read_csv(f"{build_dir}/{name}/{par}jdd.dt_ev", sep="\t")
        dt_ev[name] = dt_ev[name].iloc[:,:14]
        dt_ev[name].columns = ['time', 'dt', 'facsec', 'residu', 'dt_stab', 'vitl', 'vitg1','vitg2', 'alphal', 'alphag1', 'alphag2', 'Tl', 'Tg1','Tg2']

        sim[name] = pd.DataFrame()
        sim[name]['r+']  = np.linspace(1.0/(2*n_seg_sonde), 1 - 1.0/(2*n_seg_sonde), n_seg_sonde)
        sim[name]['vxl_88'] = (np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_LIQUIDE_EAU_881.son"))[1::dimension[s],-1]+np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_LIQUIDE_EAU_882.son"))[1::dimension[s],-1]+np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_LIQUIDE_EAU_883.son"))[1::dimension[s],-1])/3
        sim[name]['vzl_88'] = (np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_LIQUIDE_EAU_881.son"))[dimension[s]::dimension[s],-1]+np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_LIQUIDE_EAU_882.son"))[dimension[s]::dimension[s],-1]+np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_LIQUIDE_EAU_883.son"))[dimension[s]::dimension[s],-1])/3
        sim[name]['vxg1_88'] = (np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP1_881.son"))[1::dimension[s],-1]+np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP1_882.son"))[1::dimension[s],-1]+np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP1_883.son"))[1::dimension[s],-1])/3
        sim[name]['vzg1_88'] = (np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP1_881.son"))[dimension[s]::dimension[s],-1]+np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP1_882.son"))[dimension[s]::dimension[s],-1]+np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP1_883.son"))[dimension[s]::dimension[s],-1])/3
        sim[name]['vxg2_88'] = (np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP2_881.son"))[1::dimension[s],-1]+np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP2_882.son"))[1::dimension[s],-1]+np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP2_883.son"))[1::dimension[s],-1])/3
        sim[name]['vzg2_88'] = (np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP2_881.son"))[dimension[s]::dimension[s],-1]+np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP2_882.son"))[dimension[s]::dimension[s],-1]+np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP2_883.son"))[dimension[s]::dimension[s],-1])/3
        sim[name]['k_88']   = (np.array(plot.loadText(f"{name}/{par}jdd_K_881.son"))[1::,-1]+np.array(plot.loadText(f"{name}/{par}jdd_K_882.son"))[1::,-1]+np.array(plot.loadText(f"{name}/{par}jdd_K_883.son"))[1::,-1])/3
        sim[name]['alp1_88'] = (np.array(plot.loadText(f"{name}/{par}jdd_ALPHA_GAZ_AIR_GROUP1_881.son"))[1::,-1]+np.array(plot.loadText(f"{name}/{par}jdd_ALPHA_GAZ_AIR_GROUP1_882.son"))[1::,-1]+np.array(plot.loadText(f"{name}/{par}jdd_ALPHA_GAZ_AIR_GROUP1_883.son"))[1::,-1])/3
        sim[name]['alp2_88'] = (np.array(plot.loadText(f"{name}/{par}jdd_ALPHA_GAZ_AIR_GROUP2_881.son"))[1::,-1]+np.array(plot.loadText(f"{name}/{par}jdd_ALPHA_GAZ_AIR_GROUP2_882.son"))[1::,-1]+np.array(plot.loadText(f"{name}/{par}jdd_ALPHA_GAZ_AIR_GROUP2_883.son"))[1::,-1])/3
        sim[name]['p_88']   = (np.array(plot.loadText(f"{name}/{par}jdd_PRESSION_881.son"))[1::,-1]+np.array(plot.loadText(f"{name}/{par}jdd_PRESSION_882.son"))[1::,-1]++np.array(plot.loadText(f"{name}/{par}jdd_PRESSION_883.son"))[1::,-1])/3
        sim[name]['dp_88']  = (np.array(plot.loadText(f"{name}/{par}jdd_DP_881.son"))[1::,-1]+np.array(plot.loadText(f"{name}/{par}jdd_DP_882.son"))[1::,-1]+np.array(plot.loadText(f"{name}/{par}jdd_DP_883.son"))[1::,-1])/3
        sim[name]['db1_88']  = (np.array(plot.loadText(f"{name}/{par}jdd_D_BULLES_881.son"))[1::,-1][1::3]+np.array(plot.loadText(f"{name}/{par}jdd_D_BULLES_882.son"))[1::,-1][1::3]+np.array(plot.loadText(f"{name}/{par}jdd_D_BULLES_883.son"))[1::,-1][1::3])/3
        sim[name]['db2_88']  = (np.array(plot.loadText(f"{name}/{par}jdd_D_BULLES_881.son"))[1::,-1][2::3]+np.array(plot.loadText(f"{name}/{par}jdd_D_BULLES_882.son"))[1::,-1][2::3]+np.array(plot.loadText(f"{name}/{par}jdd_D_BULLES_883.son"))[1::,-1][2::3])/3
        sim[name]['ai1_88']  = (np.array(plot.loadText(f"{name}/{par}jdd_INTERFACIAL_AREA_881.son"))[1::,-1][1::3]+np.array(plot.loadText(f"{name}/{par}jdd_INTERFACIAL_AREA_882.son"))[1::,-1][1::3]+np.array(plot.loadText(f"{name}/{par}jdd_INTERFACIAL_AREA_883.son"))[1::,-1][1::3])/3
        sim[name]['ai2_88']  = (np.array(plot.loadText(f"{name}/{par}jdd_INTERFACIAL_AREA_881.son"))[1::,-1][2::3]+np.array(plot.loadText(f"{name}/{par}jdd_INTERFACIAL_AREA_882.son"))[1::,-1][2::3]+np.array(plot.loadText(f"{name}/{par}jdd_INTERFACIAL_AREA_883.son"))[1::,-1][2::3])/3
        
        sim[name]['vxl_142'] = (np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_LIQUIDE_EAU_1421.son"))[1::dimension[s],-1]+np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_LIQUIDE_EAU_1242.son"))[1::dimension[s],-1]+np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_LIQUIDE_EAU_1423.son"))[1::dimension[s],-1])/3
        sim[name]['vzl_142'] = (np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_LIQUIDE_EAU_1421.son"))[dimension[s]::dimension[s],-1]+np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_LIQUIDE_EAU_1242.son"))[dimension[s]::dimension[s],-1]+np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_LIQUIDE_EAU_1423.son"))[dimension[s]::dimension[s],-1])/3
        sim[name]['vxg1_142'] = (np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP1_1421.son"))[1::dimension[s],-1]+np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP1_1242.son"))[1::dimension[s],-1]+np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP1_1423.son"))[1::dimension[s],-1])/3
        sim[name]['vzg1_142'] = (np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP1_1421.son"))[dimension[s]::dimension[s],-1]+np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP1_1242.son"))[dimension[s]::dimension[s],-1]+np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP1_1423.son"))[dimension[s]::dimension[s],-1])/3
        sim[name]['vxg2_142'] = (np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP2_1421.son"))[1::dimension[s],-1]+np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP2_1242.son"))[1::dimension[s],-1]+np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP2_1423.son"))[1::dimension[s],-1])/3
        sim[name]['vzg2_142'] = (np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP2_1421.son"))[dimension[s]::dimension[s],-1]+np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP2_1242.son"))[dimension[s]::dimension[s],-1]+np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP2_1423.son"))[dimension[s]::dimension[s],-1])/3
        sim[name]['k_142']   = (np.array(plot.loadText(f"{name}/{par}jdd_K_1421.son"))[1::,-1]+np.array(plot.loadText(f"{name}/{par}jdd_K_1242.son"))[1::,-1]+np.array(plot.loadText(f"{name}/{par}jdd_K_1423.son"))[1::,-1])/3
        sim[name]['alp1_142'] = (np.array(plot.loadText(f"{name}/{par}jdd_ALPHA_GAZ_AIR_GROUP1_1421.son"))[1::,-1]+np.array(plot.loadText(f"{name}/{par}jdd_ALPHA_GAZ_AIR_GROUP1_1242.son"))[1::,-1]+np.array(plot.loadText(f"{name}/{par}jdd_ALPHA_GAZ_AIR_GROUP1_1423.son"))[1::,-1])/3
        sim[name]['alp2_142'] = (np.array(plot.loadText(f"{name}/{par}jdd_ALPHA_GAZ_AIR_GROUP2_1421.son"))[1::,-1]+np.array(plot.loadText(f"{name}/{par}jdd_ALPHA_GAZ_AIR_GROUP2_1242.son"))[1::,-1]+np.array(plot.loadText(f"{name}/{par}jdd_ALPHA_GAZ_AIR_GROUP2_1423.son"))[1::,-1])/3
        sim[name]['p_142']   = (np.array(plot.loadText(f"{name}/{par}jdd_PRESSION_1421.son"))[1::,-1]+np.array(plot.loadText(f"{name}/{par}jdd_PRESSION_1242.son"))[1::,-1]++np.array(plot.loadText(f"{name}/{par}jdd_PRESSION_1423.son"))[1::,-1])/3
        sim[name]['dp_142']  = (np.array(plot.loadText(f"{name}/{par}jdd_DP_1421.son"))[1::,-1]+np.array(plot.loadText(f"{name}/{par}jdd_DP_1242.son"))[1::,-1]+np.array(plot.loadText(f"{name}/{par}jdd_DP_1423.son"))[1::,-1])/3
        sim[name]['db1_142']  = (np.array(plot.loadText(f"{name}/{par}jdd_D_BULLES_1421.son"))[1::,-1][1::3]+np.array(plot.loadText(f"{name}/{par}jdd_D_BULLES_1242.son"))[1::,-1][1::3]+np.array(plot.loadText(f"{name}/{par}jdd_D_BULLES_1423.son"))[1::,-1][1::3])/3
        sim[name]['db2_142']  = (np.array(plot.loadText(f"{name}/{par}jdd_D_BULLES_1421.son"))[1::,-1][2::3]+np.array(plot.loadText(f"{name}/{par}jdd_D_BULLES_1242.son"))[1::,-1][2::3]+np.array(plot.loadText(f"{name}/{par}jdd_D_BULLES_1423.son"))[1::,-1][2::3])/3
        sim[name]['ai1_142']  = (np.array(plot.loadText(f"{name}/{par}jdd_INTERFACIAL_AREA_1421.son"))[1::,-1][1::3]+np.array(plot.loadText(f"{name}/{par}jdd_INTERFACIAL_AREA_1242.son"))[1::,-1][1::3]+np.array(plot.loadText(f"{name}/{par}jdd_INTERFACIAL_AREA_1423.son"))[1::,-1][1::3])/3
        sim[name]['ai2_142']  = (np.array(plot.loadText(f"{name}/{par}jdd_INTERFACIAL_AREA_1421.son"))[1::,-1][2::3]+np.array(plot.loadText(f"{name}/{par}jdd_INTERFACIAL_AREA_1242.son"))[1::,-1][2::3]+np.array(plot.loadText(f"{name}/{par}jdd_INTERFACIAL_AREA_1423.son"))[1::,-1][2::3])/3



# Convergence plots

In [ ]:
matplotlib.rcParams.update({'font.size': 14})

color_conf = { "VDF_ICE" : "red", "PolyMAC_ICE" : "magenta", "VDF_SETS" : "orange", "PolyMAC_SETS" : "purple"  }

fig = plt.figure(figsize = (16, 4))

name_phys = [ ["vitg1"],["vitg2"], [ "alphag1"],  [ "alphag2"]]

axs = fig.subplots(1, len(name_phys) )#, sharex=True, sharey=True)

for l in range(len(name_phys)): #phys
    for r in dt_ev.keys() :
        for lp in range(len(name_phys[l])) : #physique de la phase si multiphase
            axs[l].plot(dt_ev[r]["time"][::], dt_ev[r][name_phys[l][lp]][::], "-", label = r.replace("_", " "), color = color_conf[r])

        axs[l].set_xlim(0,tmax)
        axs[l].set_ylim(float(seuil_statio)/100, 1e4)
        axs[l].set_yscale("log")
        axs[l].set_title("convergence "+name_phys[l][0] )
        axs[l].set_xlabel('time')
        axs[l].set_ylabel("convergence " + name_phys[l][0])
        axs[l].legend(prop={'size': 10})

plt.tight_layout()

# Void fraction and velocities plots

In [ ]:
# experimental data

R= [0.013*100, 0.05*100 , 0.15*100, 0.25*100, 0.35*100, 0.45*100]
Al1_88  = [0.095,0.081,0.079,0.072,0.072,0.074]
Al1_142 = [0.130,0.098,0.083,0.077,0.076,0.072]
Al2_88 = [ 0.012,0.092,0.195,0.218,0.241,0.246]
Al2_142 = [0.002,0.048,0.198,0.252,0.269,0.290]


Vg1_88  = [ 3.80,4.42,4.70,4.81,5.09,5.34 ]
Vg1_142 = [ 4.04,4.55,5.03,5.21,5.30,5.45 ]
Vg2_88 = [2.69,3.94,4.40,4.52,5.02,5.21 ]
Vg2_142 = [ 2.78, 4.09,4.81,4.84,4.95,5.05 ]


# Colonnes : Titre, alpha, d, U (contient U_l et U_g), u'l
  
color_group1 = "darkorchid"
color_group2 = "royalblue"

marker_conf =  { "PolyMAC_ICE" : "--" }

plt.figure(figsize = (12,6))

plt.subplot(221)
plt.plot(R, Al1_88, "^", color = color_group1,  label = "Exp z/d=88")
plt.plot(R, Al1_142, "o", color = color_group1,  label = "Exp z/d=142")


for s in dt_ev.keys():
        f = os.open(f"{build_dir}/{s}/{par}jdd.perf", os.O_RDONLY)
        plt.plot(np.array(sim[s]["r+"])*0.100*1000, sim[s]['alp1_88'], marker_conf[s], linewidth = 3, label = f"Simu 88 {s.replace('_',' ')} ", color = "blue")
        plt.plot(np.array(sim[s]["r+"])*0.100*1000, sim[s]['alp1_142'], marker_conf[s], linewidth = 3, label = f"Simu 142 {s.replace('_',' ')} ", color = "red")

plt.legend(fontsize = 8)
plt.xlabel("Transversal position (mm)")
plt.ylabel("Void fraction")
plt.title("Comparison between experimental data and Group 1 results")
    
plt.subplot(222)

plt.plot(R, Al2_88, "^", color = color_group2,  label = "Exp z/d=88")
plt.plot(R, Al2_142, "o", color = color_group2,  label = "Exp z/d=142")

for s in dt_ev.keys():
        f = os.open(f"{build_dir}/{s}/{par}jdd.perf", os.O_RDONLY)
        plt.plot(np.array(sim[s]["r+"])*0.100*1000, sim[s]['alp2_88'], marker_conf[s], linewidth = 3, label = f"Simu 88 {s.replace('_',' ')} ", color = "blue")
        plt.plot(np.array(sim[s]["r+"])*0.100*1000, sim[s]['alp2_142'], marker_conf[s], linewidth = 3, label = f"Simu 142 {s.replace('_',' ')} ", color = "red")

plt.legend(fontsize = 8)
plt.xlabel("Transversal position (mm)")
plt.ylabel("Void fraction")
plt.title("Comparison between experimental data and Group 2 results")

plt.subplot(223)
plt.plot(R, Vg1_88, "^", color = color_group1,  label = "Exp z/d=88")
plt.plot(R, Vg1_142, "o", color = color_group1,  label = "Exp z/d=142")

for s in dt_ev.keys():
        f = os.open(f"{build_dir}/{s}/{par}jdd.perf", os.O_RDONLY)
        plt.plot(np.array(sim[s]["r+"])*0.100*1000, sim[s]['vzg1_88'], marker_conf[s], linewidth = 3, label = f"Simu 88 {s.replace('_',' ')} ", color = "blue")
        plt.plot(np.array(sim[s]["r+"])*0.100*1000, sim[s]['vzg1_142'], marker_conf[s], linewidth = 3, label = f"Simu 142 {s.replace('_',' ')} ", color = "red")
plt.legend(fontsize = 8)
plt.xlabel("Transversal position (mm)")
plt.ylabel("Gas velocity")

plt.subplot(224)
plt.plot(R, Vg2_88, "^", color = color_group2,  label = "Exp z/d=88")
plt.plot(R, Vg2_142, "o", color = color_group2,  label = "Exp z/d=142")

for s in dt_ev.keys():
        f = os.open(f"{build_dir}/{s}/{par}jdd.perf", os.O_RDONLY)
        plt.plot(np.array(sim[s]["r+"])*0.100*1000, sim[s]['vzg2_88'], marker_conf[s], linewidth = 3, label = f"Simu 88 {s.replace('_',' ')} ", color = "blue")
        plt.plot(np.array(sim[s]["r+"])*0.100*1000, sim[s]['vzg2_142'], marker_conf[s], linewidth = 3, label = f"Simu 142 {s.replace('_',' ')} ", color = "red")

plt.legend(fontsize = 8)
plt.xlabel("Transversal position (mm)")
plt.ylabel("Gas velocity")


plt.tight_layout()

# Interfacial area concentration plots

In [ ]:
#Experimental data 
Ai1_88  = [377.3,268.9,229.8,203.1,180.0,183.6]
Ai1_142 = [482.5,351.5,255.5,222.3,211.8,199.6 ]
Ai2_88 = [8.8,28.1,52.6,61.4,63.2,71.9 ]
Ai2_142 = [2.2,16.2,47.6,61.6,70.3, 66.8 ] 

# Colonnes : Titre, alpha, d, U (contient U_l et U_g), u'l
    
color_group1 = "darkorchid"
color_group2 = "royalblue"
marker_conf =  { "PolyMAC_ICE" : "--" }

plt.figure(figsize = (12,6))

plt.subplot(121)
plt.plot(R, Ai1_88, "^", color = color_group1,  label = "Exp z/d=88")
plt.plot(R, Ai1_142, "o", color = color_group1,  label = "Exp z/d=142")
for s in dt_ev.keys():
        f = os.open(f"{build_dir}/{s}/{par}jdd.perf", os.O_RDONLY)
        plt.plot(np.array(sim[s]["r+"])*0.100*1000, sim[s]['ai1_88'], marker_conf[s], linewidth = 3, label = f"Simu 88 {s.replace('_',' ')} ", color = "blue")
        plt.plot(np.array(sim[s]["r+"])*0.100*1000, sim[s]['ai1_142'], marker_conf[s], linewidth = 3, label = f"Simu 142 {s.replace('_',' ')} ", color = "red")

plt.legend(fontsize = 8)
plt.xlabel("Transversal position (mm)")
plt.ylabel(r"Interfacial area concentration group 1 ($m^{-1}$)")
plt.title("Interfacial area concentration")
    
plt.subplot(122)
plt.plot(R, Ai2_88, "^", color = color_group2,  label = "Exp z/d=88")
plt.plot(R, Ai2_142, "o", color = color_group2,  label = "Exp z/d=142")
for s in dt_ev.keys():
        f = os.open(f"{build_dir}/{s}/{par}jdd.perf", os.O_RDONLY)
        plt.plot(np.array(sim[s]["r+"])*0.100*1000, sim[s]['ai2_88'], marker_conf[s], linewidth = 3, label = f"Simu 88 {s.replace('_',' ')} ", color = "blue")
        plt.plot(np.array(sim[s]["r+"])*0.100*1000, sim[s]['ai2_142'], marker_conf[s], linewidth = 3, label = f"Simu 142 {s.replace('_',' ')} ", color = "red")

plt.legend(fontsize = 8)
plt.xlabel("Transversal position (mm)")
plt.ylabel(r"Interfacial area concentration group 2 ($m^{-1}$)")
plt.title("Interfacial area concentration")

plt.tight_layout()

# Bubble diameter plots

In [ ]:
#Experimental data 
Dsm1_88  = [6.*Al1_88[i]/Ai1_88[i]*1000 for i in range(len(Al1_88))]
Dsm1_142 = [6.*Al1_142[i]/Ai1_142[i]*1000 for i in range(len(Al1_142))]
Dsm2_88 = [6.*Al2_88[i]/Ai2_88[i]*1000 for i in range(len(Al2_88))]
Dsm2_142 = [6.*Al2_142[i]/Ai2_142[i]*1000 for i in range(len(Al2_142))] 

# Colonnes : Titre, alpha, d, U (contient U_l et U_g), u'l
    
color_group1 = "darkorchid"
color_group2 = "royalblue"
marker_conf =  {  "PolyMAC_ICE" : "--" }

plt.figure(figsize = (12,6))

plt.subplot(121)
plt.plot(R, Dsm1_88, "^", color = color_group1,  label = "Exp z/d=88")
plt.plot(R, Dsm1_142, "o", color = color_group1,  label = "Exp z/d=142")

for s in dt_ev.keys():
        f = os.open(f"{build_dir}/{s}/{par}jdd.perf", os.O_RDONLY)
        plt.plot(np.array(sim[s]["r+"])*0.100*1000, sim[s]['db1_88']*1000, marker_conf[s], linewidth = 3, label = f"Simu 88 {s.replace('_',' ')} ", color = "blue")
        plt.plot(np.array(sim[s]["r+"])*0.100*1000, sim[s]['db1_142']*1000, marker_conf[s], linewidth = 3, label = f"Simu 142 {s.replace('_',' ')} ", color = "red")


plt.legend(fontsize = 8)
plt.xlabel("Transversal position (mm)")
plt.ylabel("Bubble diameter group 1 (mm)")
plt.title("Group 1 bubble diameter")
    
plt.subplot(122)
plt.plot(R, Dsm2_88, "^", color = color_group2,  label = "Exp z/d=88")
plt.plot(R, Dsm2_142, "o", color = color_group2,  label = "Exp z/d=142")
for s in dt_ev.keys():
        f = os.open(f"{build_dir}/{s}/{par}jdd.perf", os.O_RDONLY)
        plt.plot(np.array(sim[s]["r+"])*0.100*1000, sim[s]['db2_88']*1000, marker_conf[s], linewidth = 3, label = f"Simu 88 {s.replace('_',' ')} ", color = "blue")
        plt.plot(np.array(sim[s]["r+"])*0.100*1000, sim[s]['db2_142']*1000, marker_conf[s], linewidth = 3, label = f"Simu 142 {s.replace('_',' ')} ", color = "red")

plt.legend(fontsize = 8)
plt.xlabel("Transversal position (mm)")
plt.ylabel("Bubble diameter group 2 (mm)")
plt.title("Group 2 bubble diameter ")

plt.tight_layout()